In [3]:
import re


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2025/{day}-example.txt" if example else f"2025/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [4]:
# Day 1: Secret Entrance
def rotate(rotations, dial=50):
    stops_at_zero, passes_zero = 0, 0
    for rotation in rotations:
        dir, steps = rotation[0], int(rotation[1:])
        passes_zero += steps // 100  # Full rotations
        if dir == "R":
            new_dial = (dial + steps) % 100
            if new_dial < dial and new_dial != 0:
                passes_zero += 1
        else:
            new_dial = (dial - steps) % 100
            if new_dial > dial and dial != 0:
                passes_zero += 1
        if new_dial == 0:
            stops_at_zero += 1
        dial = new_dial
    return stops_at_zero, passes_zero


stops, passes = rotate(data(1))
print(f"Part 1: {stops}")  # 995
print(f"Part 2: {stops + passes}")  # 5847

Part 1: 995
Part 2: 5847


In [5]:
# Day 2: Gift Shop
def invalid_ids(ranges, part1=True):
    def invalid(id):
        id = str(id)
        middle = len(id) // 2
        first_half = id[:middle]
        second_half = id[middle:]
        return first_half == second_half

    def invalid_part2(id):
        # Substrings from pos 1 -> middle, check if they are repeated for the
        # rest of the string
        id = str(id)
        for i in range(1, len(id) // 2 + 1):
            if len(id) % i == 0:
                repetitions = len(id) // i - 1
                if id[:i] * repetitions == id[i:]:
                    return True  # repeated pattern
        return False

    def start_stop(_range):
        start, stop = _range.split("-")
        return range(int(start), int(stop) + 1)

    if part1:
        return sum(id for _range in ranges for id in start_stop(_range) if invalid(id))
    else:
        return sum(
            id for _range in ranges for id in start_stop(_range) if invalid_part2(id)
        )


print(f"Part 1: {invalid_ids(data(2, sep=','))}")  # 56660955519
print(f"Part 2: {invalid_ids(data(2, sep=','), part1=False)}")  # 79183223243

Part 1: 56660955519
Part 2: 79183223243


In [6]:
# Day 3: Lobby
def max_jolt(bank, batteries_following, start_position=0):
    # find largest jolt value in bank from start_position with number of
    # batteries_following. Returns largest jolt and next start_position to start
    # from.
    jolt, position = 0, 0
    for pos, battery in enumerate(bank[start_position:], start_position):
        if int(battery) > jolt and pos + batteries_following < len(bank):
            jolt = int(battery)
            position = pos
    return jolt, position + 1


def max_battery(bank, number_of_batteries=2):
    result, start = 0, 0
    for batteries_following in range(number_of_batteries - 1, -1, -1):
        jolt, start = max_jolt(bank, batteries_following, start)
        result += 10**batteries_following * jolt
    return result


banks = data(3)
print(f"Part 1: {sum(max_battery(bank) for bank in banks)}")  # 17087
print(f"Part 2: {sum(max_battery(bank, 12) for bank in banks)}")  # 169019504359949


Part 1: 17087
Part 2: 169019504359949


In [7]:
# Day 4: Printing Department
def neighbors(point, grid):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for r, c in potential:
        if 0 <= r < len(grid[0]) and 0 <= c < len(grid):
            yield (r, c)


def adjacent_paper_rolls(point, grid):
    return sum(1 for p in neighbors(point, grid) if grid[p[0]][p[1]] == "@")


def removable_rolls(grid):
    result = []
    for r, _ in enumerate(grid):
        for c, char in enumerate(grid[r]):
            p = (r, c)
            if char == "@" and adjacent_paper_rolls(p, grid) < 4:
                result += [p]
    return result


def remove_roll(point, grid):
    r, c = point
    row = list(grid[r])
    row[c] = "."
    grid[r] = "".join(row)
    return grid


def one_forklift_round(grid):
    removable = removable_rolls(grid)
    for p in removable:
        grid = remove_roll(p, grid)
    return len(removable), grid


grid = data(4)

print(f"Part 1: {len(removable_rolls(grid))}")  # 1346

rolls_removed_this_round, grid = one_forklift_round(grid)
total_rolls_removed = rolls_removed_this_round
while rolls_removed_this_round > 0:
    rolls_removed_this_round, grid = one_forklift_round(grid)
    total_rolls_removed += rolls_removed_this_round

print(f"Part 2: {total_rolls_removed}")  # 8493

Part 1: 1346
Part 2: 8493


In [14]:
# Day 5: Cafeteria
def within_range(id, _range):
    low, high = _range.split("-")
    low, high = int(low), int(high)
    return low <= id <= high


def is_fresh(id, ranges):
    return any(within_range(id, r) for r in ranges)


fresh_ranges, available_ids = data(5, sep="\n\n")
fresh_ranges = fresh_ranges.splitlines()
available_ids = ints(available_ids)
print(f"Part 1: {sum(1 for id in available_ids if is_fresh(id, fresh_ranges))}")  # 744


Part 1: 744


In [ ]:
example = """3-5
10-14
16-20
12-18

1
5
8
11
17
32"""

fresh_ranges, available_ids = example.split("\n\n")
# fresh_ranges, available_ids = data(5, sep="\n\n")
fresh_ranges = fresh_ranges.splitlines()


def overlapping_ranges(ranges):
    lows, highs = [], []
    for r in ranges:
        low, high = r.split("-")
        low = int(low)
        high = int(high)
        lows.append(low)
        highs.append(high)
    print(lows, highs)


overlapping_ranges(fresh_ranges)


[3, 10, 16, 12] [5, 14, 20, 18]


14